In [2]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [3]:
def enum_other_states(n, num_step):
    """enumerate the state. using the following invariants
    (1) symetry (.1,.1,.8) is equivalent to (.1,.1,.8) from firm 1's pespective
    (2) the states must sum to 1"""
    
    states_i = np.arange(0,num_step+1)*(1./num_step)
    if n <=1:
        return states_i.reshape(num_step+1,1)
    else:
        states = []
        states_j = enum_other_states(n-1, num_step)
        for i in states_i:
            for j in states_j:
                if j[0] >= i and (j.sum() + i <= 1):
                    state_ij = np.concatenate( ([i],j) )
                    states.append(state_ij)
        states = np.array(states)
        return  states
    
    
def make_3d(np_array):
    """make a 2d np array 3d"""
    return np.array(np_array).reshape((len(np_array),1))

In [10]:
class States:
    """class for dealing with the states to ensure
    invariants are kept"""
    
    my_states = None #type nparray
    other_states = None #type np array 2d
    n = 0 #number of players
    num_step = 0 #number of states (states normalized between 0 and 1)
    types = None

    def __init__(self, n, num_step):
        """initialize important class attributes"""
        self.n, self.num_step = n, num_step
      
    
    def gen_states(self):
        """compute all relevant states for firm i (Avoiding side effects)
        the trick is enforcing invariants in the helper"""
        
        new_states = States(self.n, self.num_step)
        other_states =  enum_other_states(self.n-1, self.num_step)
        my_states = 1 - other_states.sum(axis=1)
        
        #initialize the 'type'
        new_states.types = np.tile([0,1],len(my_states))
        new_states.my_states = np.repeat(my_states,2)
        new_states.other_states = np.repeat(other_states,2,axis=0)
        
        return new_states
    
    
    def get_all(self,  types=False, filter_types=None, order=False):
        """return various attributes as an array"""
        other_states = self.other_states
        if order: other_states = np.sort(self.other_states)
        
        reshaped_states = make_3d(self.my_states)
        all_states = np.concatenate( (reshaped_states, other_states), axis=1 )
        
        #add types if needed
        if types:
            reshaped_types = make_3d(self.types)
            all_states = np.concatenate( (all_states,reshaped_types), axis=1 )
        
        #filter certain types
        if filter_types == None:
            return all_states
        else:
            return all_states[self.types == types ]
    
    
    def get_len(self):
        """qaulity of life, len of states"""
        return len(self.other_states)
    
    
    def make_states(self, my_states, other_states, types):
        """special initializer for states enforcing the invariants"""
        assert np.array(other_states).shape[1] == (self.n -1)
        #make a copy to avoid side effects
        new_states = States(self.n, self.num_step)
        
        #combine into 1 array
        my_states = make_3d(my_states)
        all_states = np.concatenate((my_states,other_states),axis=1)
        all_states = np.maximum(0, all_states )

        #re-weight states
        denom = all_states.sum(axis=1)
        denom = np.repeat(denom,self.n).reshape(all_states.shape)
        all_states = all_states/denom
        
        #return states
        new_states.types = np.clip(np.array(types).astype(int),0,1)
        new_states.my_states = all_states[:,0]
        new_states.other_states = np.delete(all_states,0,axis=1)
        
        return new_states

In [11]:
def set_up_env(n, nsim, steps, params):
    """generate reasonable parameters for simulations
    
    theta0 - mean costs; 
    theta1 - returns to scale
    theta2 - lower cost
    theta3 - drs
    
    we need to have theta0 < theta2
    
    theta4 - forgivness probability
    theta5 - size of contracts """
    
    states = States(n, steps).gen_states()
    
    #set up shocks
    var_cost = 1.2/steps
    cost_shocks = np.random.normal(0,var_cost,(nsim,n))
    type_shocks = np.random.binomial(1, params[4], size=(nsim,1))
    shocks = np.concatenate((cost_shocks,type_shocks),axis=1)
    
    #set up bids
    bsteps = int(1.1*steps)
    min_bid = params[2] - 2*var_cost
    max_bid = params[0] + params[1] + 3*var_cost
    bid_weighting = (max_bid - min_bid)/(bsteps)
    bids = min_bid + np.arange(1,bsteps)*bid_weighting

    policy = lambda state, shock: 2*state.my_states + params[2]*state.types + shock[:,0]
    value = lambda state: 1 - state.my_states + state.types
    
    return value, policy, bids, states, shocks, params


params =  2.5, -.3, 2., .2, .1, .1, .9 #beta comes lasts
value, policy, bids, states, shocks, params = set_up_env(3, 5, 3, params)

In [17]:
def cost(states, shocks, params):
    """return the cost for all firms based on states"""
    all_types = make_3d(states.types)
    all_types = np.tile(all_types, (1,states.n))
    
    c1 = params[0] + params[1]*states.get_all()
    c2 = params[2] + params[3]*states.get_all()
    
    return c1*(1-all_types) + c2*(all_types) + shocks[:,:-1]


def weighting(bids):
    """return weighting when there's a tie"""
    n_states, n_players = bids.shape
    low_bid =  np.min(bids,axis=1)
    low_bid = np.repeat(low_bid, n_players).reshape(n_states,n_players)
    
    #winners
    win = (bids <= low_bid)
    n_winners = win.sum(axis=1)*1.
    n_winners = np.repeat(n_winners, n_players).reshape(n_states,n_players)
    
    #add loosers
    lose = (bids > low_bid)
    n_loosers = np.maximum(lose.sum(axis=1)*1.,1.)
    n_loosers = np.repeat(n_loosers, n_players).reshape(n_states,n_players)
    
    return win/(1.*n_winners), lose/(1.*n_loosers)


def profit(bids, states, shocks, params):
    """return firm 0's profit"""
    bids,shocks = np.array(bids),np.array(shocks)
    win_weights, lose_weights = weighting(bids)
    profits = params[5]*win_weights*(bids - cost(states,shocks,params))  
    return profits


def update_state(bids, states, shocks, params):
    """update all firms states based on bids
    note: firms in updated states, no longer correspond to their bids"""
    bids,shocks = np.array(bids),np.array(shocks)
    
    win_weights, lose_weights = weighting(bids)
    new_states = params[5]*win_weights - params[5]*lose_weights + states.get_all()
    new_types = np.maximum(states.types,shocks[:,-1])
    return states.make_states(new_states[:,0],np.delete(new_states,0,axis=1), new_types)


####unit testing######


params =  2.5, -.3, 2., .2, .1, .1, .9 #beta comes lasts
states = States(3,4)
new_states = states.make_states([.6,.6],[[.2,.2],[.2,.2]],[0,1])
new_shocks = np.array([[0,0,0,0],[0,0,0,0]])


print cost(new_states, new_shocks, params)
print profit([[2,1,2],[2,1,2]], new_states, new_shocks, params)
print update_state([[2,1,2],[2,1,2]], new_states, new_shocks, params).get_all(types=True)

[[2.32 2.44 2.44]
 [2.12 2.04 2.04]]
[[-0.    -0.144 -0.   ]
 [-0.    -0.104 -0.   ]]
[[0.55 0.3  0.15 0.  ]
 [0.55 0.3  0.15 1.  ]]
